# Dev

In [1]:
from pyspark.sql import SparkSession
import numpy as np
import h5py
import pydoop.hdfs as hdfs

In [6]:
# Create a Spark session
spark = SparkSession.builder \
    .appName("Pydoop_experiments_Rustam1") \
    .master("yarn") \
    .config("spark.executor.instances", "2") \
    .config("spark.yarn.appMasterEnv.PYSPARK_PYTHON", "/usr/bin/python3") \
    .config("spark.executorEnv.PYSPARK_PYTHON", "/usr/bin/python3") \
    .getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Application application_1709508717695_0069 was killed by user dr.who
	at org.apache.spark.scheduler.cluster.YarnClientSchedulerBackend.waitForApplication(YarnClientSchedulerBackend.scala:98)
	at org.apache.spark.scheduler.cluster.YarnClientSchedulerBackend.start(YarnClientSchedulerBackend.scala:65)
	at org.apache.spark.scheduler.TaskSchedulerImpl.start(TaskSchedulerImpl.scala:235)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:604)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:

In [4]:
sc = spark.sparkContext

# Define your HDFS directory path
directory_path = "hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A"

NameError: name 'spark' is not defined

In [ ]:
def get_h5_file_paths_hdfs(directory_path):
    """
    List all HDF5 (.h5) file paths in the specified HDFS directory.
    """
    h5_file_paths = []
    fs = hdfs.hdfs()  # Get a reference to the HDFS file system

    try:
        entries = fs.list_directory(directory_path)
        for entry in entries:
            entry_path = entry['name']
            if entry['kind'] == 'directory':
                # Recursive call for directories
                h5_file_paths.extend(get_h5_file_paths_hdfs(entry_path))
            elif entry_path.endswith(".h5"):
                # Add the HDF5 file path to the list
                h5_file_paths.append(entry_path)
    finally:
        fs.close()  # Ensure the file system reference is closed properly

    return h5_file_paths

def extract_info_pyd(hdfs_path):
    """
    Extract selected features from an HDF5 file stored in HDFS.
    """
    with hdfs.open(hdfs_path, 'rb') as f:  # Ensure binary mode for h5py compatibility
        with h5py.File(f, 'r', driver='fileobj') as h:
            dataset = h['/analysis/songs']
            np_dataset = np.array(dataset[0])
            
            selected_features = np_dataset[['danceability', 'duration', 'tempo', 'energy', 'loudness']].ravel()
            return selected_features.tolist()

In [ ]:
# Use the function to get HDF5 file paths from HDFS
h5_file_paths = get_h5_file_paths_hdfs(directory_path)

print(f'{len(h5_file_paths)} HDF5 file paths gathered (HDFS)')
for file_path in h5_file_paths[:5]:  # Print a few paths as a sample
    print(file_path)

# Parallelize the file paths as an RDD
file_paths_rdd = sc.parallelize(h5_file_paths[:2])

# Use Pydoop and h5py to extract information from each HDF5 file in HDFS
files_infos_rdd = file_paths_rdd.map(extract_info_pyd)

In [ ]:
# Collect and print results (for demonstration; be mindful of memory constraints with large datasets)
print(files_infos_rdd.collect())

In [5]:
# release the cores
spark.stop()

# Backup

In [1]:
#PYTHON
import h5py
import numpy as np
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyspark

#SPARK
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *
from operator import add

Spark Configuration

In [2]:
spark_session = SparkSession.builder \
    .appName("Pydoop_experiments_Rustam") \
    .master("yarn") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

sqlContext = SQLContext(spark_session.sparkContext)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/08 19:44:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/08 19:44:16 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
/home/hadoop/.local/lib/python3.8/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


File Path

In [3]:
#file_roots = ['hdfs://130.238.29.183:9000//input/millionsongsubset.tar.gz']
#directory_path = "data/B/A/A"
#directory_path = "data_AZ"
#Path300
#directory_path = "/user/hadoop/MillionSongSubset/A/A/"
#directory_path = "/user/hadoop/MillionSongSubset/A/A/A"
directory_path = "hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A"

In [42]:
# # Use Spark to list files in HDFS directory
# hdfs_files = spark_session._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()) \
#     .listStatus(spark_session._jvm.org.apache.hadoop.fs.Path(directory_path))

# 1) Saving the paths to the HDF5 files

In [4]:
def get_h5_file_paths_hdfs(spark, hdfs_directory_path):
    h5_file_paths = []

    # Use Spark to list files in HDFS directory
    hdfs_files = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()) \
        .listStatus(spark._jvm.org.apache.hadoop.fs.Path(hdfs_directory_path))

    # Extract file paths from HDFS file status
    for hdfs_file_status in hdfs_files:
        file_path = hdfs_file_status.getPath().toString()

        if hdfs_file_status.isDirectory():
            # If it's a directory, recursively gather paths
            h5_file_paths.extend(get_h5_file_paths_hdfs(spark, file_path))
        elif file_path.endswith(".h5"):
            # If it's an HDF5 file, add to the list
            h5_file_paths.append(file_path)

    return h5_file_paths

In [5]:
h5_file_paths = get_h5_file_paths_hdfs(spark_session, directory_path)

print('{} HDF5 file paths gathered (HDFS)'.format(len(h5_file_paths)))
for file_path in h5_file_paths[:50]:
    print(file_path)

11 HDF5 file paths gathered (HDFS)
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAABD128F429CF47.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAADZ128F9348C2E.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAEF128F4273421.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAFD128F92F423A.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAMO128F1481E7F.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAMQ128F1460CD3.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAPK128E0786D96.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAARJ128F9320760.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAVG12903CFA543.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAVO128F93133D4.h5


# 2) List h5_file_paths is converted to RDD

In [6]:
h5_file_paths[:30]

['hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAABD128F429CF47.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAADZ128F9348C2E.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAEF128F4273421.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAFD128F92F423A.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAMO128F1481E7F.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAMQ128F1460CD3.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAPK128E0786D96.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAARJ128F9320760.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAVG12903CFA543.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAVO128F93133D4.h5']

In [7]:
file_paths = spark_context.parallelize(h5_file_paths)
file_paths

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

In [8]:
file_first = file_paths.first()
file_first

24/03/08 19:44:59 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/08 19:44:59 INFO DAGScheduler: Got job 0 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/08 19:44:59 INFO DAGScheduler: Final stage: ResultStage 0 (runJob at PythonRDD.scala:181)
24/03/08 19:44:59 INFO DAGScheduler: Parents of final stage: List()
24/03/08 19:44:59 INFO DAGScheduler: Missing parents: List()
24/03/08 19:44:59 INFO DAGScheduler: Submitting ResultStage 0 (PythonRDD[1] at RDD at PythonRDD.scala:53), which has no missing parents
24/03/08 19:44:59 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 5.9 KiB, free 366.3 MiB)
24/03/08 19:44:59 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 3.8 KiB, free 366.3 MiB)
24/03/08 19:44:59 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on master-node:41023 (size: 3.8 KiB, free: 366.3 MiB)
24/03/08 19:44:59 INFO SparkContext: Created broadcast 0 from broadcast at

'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5'

# 3) H5Py file object file creation from the H5 file 

In [9]:
import subprocess

In [10]:
def extract_info_using_h5dump(file_path):
    # Verwende h5dump, um die Daten aus der HDF5-Datei zu extrahieren
    command = ["h5dump", "-d", "/analysis/songs", file_path]
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Hier musst du den result.stdout-Text analysieren und die relevanten Informationen extrahieren
    # Du kannst regex, string splitting, oder andere Methoden verwenden, um die benötigten Daten zu isolieren
    extracted_info = parse_h5dump_output(result.stdout)
    
    return extracted_info

def parse_h5dump_output(output):
    # Beispiel: Annahme, dass die relevanten Informationen als Key-Value-Paare vorliegen
    # Hier musst du deine eigene Logik anpassen, basierend auf dem tatsächlichen h5dump-Ausgabeformat
    lines = output.split('\n')
    info_dict = {}
    for line in lines:
        if ':' in line:
            key, value = map(str.strip, line.split(':'))
            info_dict[key] = value
    
    return info_dict

In [13]:
from pydoop import hdfs

def extract_info_pyd(hdfs_path):
    with hdfs.open(hdfs_path) as f:
        with h5py.File(f, driver='fileobj') as h:
            # Assuming the dataset path is '/analysis/songs'
            dataset = h['/analysis/songs']
            np_dataset = np.array(dataset[0])
            
            # Extracting selected features
            selected_features = np_dataset[['danceability', 'duration', 'tempo', 'energy', 'loudness']].ravel()
            
            return selected_features.tolist()

In [14]:
def extract_info(file_path):
    with h5py.File(file_path, 'r') as file:
        # Informationen extrahieren
        song = file['analysis']['songs']
        np_song = np.array(song[0])
        selected_features = np_song[['danceability', 'duration', 'tempo', 'energy', 'loudness']].ravel()
        return selected_features.tolist()

In [11]:
#file = h5py.File(file_first)
files_infos_rdd = file_paths.map(extract_info_using_h5dump)

In [12]:
print(files_infos_rdd.collect())

24/03/08 19:58:11 INFO SparkContext: Starting job: collect at /tmp/ipykernel_223526/3743736607.py:1
24/03/08 19:58:11 INFO DAGScheduler: Got job 1 (collect at /tmp/ipykernel_223526/3743736607.py:1) with 2 output partitions
24/03/08 19:58:11 INFO DAGScheduler: Final stage: ResultStage 1 (collect at /tmp/ipykernel_223526/3743736607.py:1)
24/03/08 19:58:11 INFO DAGScheduler: Parents of final stage: List()
24/03/08 19:58:11 INFO DAGScheduler: Missing parents: List()
24/03/08 19:58:11 INFO DAGScheduler: Submitting ResultStage 1 (PythonRDD[2] at collect at /tmp/ipykernel_223526/3743736607.py:1), which has no missing parents
24/03/08 19:58:11 INFO MemoryStore: Block broadcast_1 stored as values in memory (estimated size 6.2 KiB, free 366.3 MiB)
24/03/08 19:58:11 INFO MemoryStore: Block broadcast_1_piece0 stored as bytes in memory (estimated size 4.1 KiB, free 366.3 MiB)
24/03/08 19:58:11 INFO BlockManagerInfo: Added broadcast_1_piece0 in memory on master-node:41023 (size: 4.1 KiB, free: 366.3

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 4 times, most recent failure: Lost task 0.3 in stage 1.0 (TID 5) (worker-node-2 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-hadoop/nm-local-dir/usercache/hadoop/appcache/application_1709508717695_0054/container_1709508717695_0054_01_000002/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/tmp/hadoop-hadoop/nm-local-dir/usercache/hadoop/appcache/application_1709508717695_0054/container_1709508717695_0054_01_000002/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/tmp/hadoop-hadoop/nm-local-dir/usercache/hadoop/appcache/application_1709508717695_0054/container_1709508717695_0054_01_000002/pyspark.zip/pyspark/serializers.py", line 274, in dump

In [15]:
h5py.run_tests()

Tests require pytest, pytest not installed


1

# 4) Create Spark Data Frame

In [61]:
# Definiere das Schema für den DataFrame
schema = StructType([
    StructField("danceability", FloatType(), True),
    StructField("duration", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("loudness", FloatType(), True),
])

In [62]:
spark_df = sqlContext.createDataFrame(files_infos_rdd.flatMap(lambda x: x), schema=schema)

24/03/08 14:24:48 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/03/08 14:24:48 INFO SharedState: Warehouse path is 'file:/home/hadoop/MSD_Project/DataEngineeringProject/Basic%20analysis%20hdf5_to_rdd/spark-warehouse'.
24/03/08 14:24:48 INFO ServerInfo: Adding filter to /SQL: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/08 14:24:48 INFO ServerInfo: Adding filter to /SQL/json: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/08 14:24:48 INFO ServerInfo: Adding filter to /SQL/execution: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/08 14:24:48 INFO ServerInfo: Adding filter to /SQL/execution/json: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/08 14:24:48 INFO ServerInfo: Adding filter to /static/sql: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/08 14:24:49 INFO BlockManagerInfo: Removed broadcast_2_piece0 on master-node:43017 in memory

In [63]:
# Zeige den DataFrame-Inhalt
spark_df.show()

24/03/08 14:24:56 INFO CodeGenerator: Code generated in 283.099102 ms
24/03/08 14:24:56 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/03/08 14:24:56 INFO DAGScheduler: Got job 3 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/03/08 14:24:56 INFO DAGScheduler: Final stage: ResultStage 3 (showString at NativeMethodAccessorImpl.java:0)
24/03/08 14:24:56 INFO DAGScheduler: Parents of final stage: List()
24/03/08 14:24:56 INFO DAGScheduler: Missing parents: List()
24/03/08 14:24:56 INFO DAGScheduler: Submitting ResultStage 3 (MapPartitionsRDD[9] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/08 14:24:56 INFO MemoryStore: Block broadcast_3 stored as values in memory (estimated size 22.5 KiB, free 366.3 MiB)
24/03/08 14:24:56 INFO MemoryStore: Block broadcast_3_piece0 stored as bytes in memory (estimated size 10.4 KiB, free 366.3 MiB)
24/03/08 14:24:56 INFO BlockManagerInfo: Added broadcast_

Py4JJavaError: An error occurred while calling o62012.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 4 times, most recent failure: Lost task 0.3 in stage 3.0 (TID 13) (worker-node-2 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-hadoop/nm-local-dir/usercache/hadoop/appcache/application_1709508717695_0037/container_1709508717695_0037_01_000002/pyspark.zip/pyspark/worker.py", line 1227, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/tmp/hadoop-hadoop/nm-local-dir/usercache/hadoop/appcache/application_1709508717695_0037/container_1709508717695_0037_01_000002/pyspark.zip/pyspark/worker.py", line 90, in read_command
    command = serializer._read_with_length(file)
  File "/tmp/hadoop-hadoop/nm-local-dir/usercache/hadoop/appcache/application_1709508717695_0037/container_1709508717695_0037_01_000002/pyspark.zip/pyspark/

# Calculation (Average)

In [15]:
average_values = spark_df.agg({'danceability': 'avg', 'duration': 'avg', 'tempo': 'avg', 'energy': 'avg', 'loudness': 'avg'}).collect()[0]
print("Average Values::")
for col_name, avg_value in average_values.asDict().items():
    print(f"{col_name}: {avg_value}")

24/03/01 12:54:24 INFO CodeGenerator: Code generated in 159.97677 ms
24/03/01 12:54:24 INFO DAGScheduler: Registering RDD 10 (collect at /tmp/ipykernel_56411/3998951206.py:1) as input to shuffle 0
24/03/01 12:54:24 INFO DAGScheduler: Got map stage job 4 (collect at /tmp/ipykernel_56411/3998951206.py:1) with 2 output partitions
24/03/01 12:54:24 INFO DAGScheduler: Final stage: ShuffleMapStage 4 (collect at /tmp/ipykernel_56411/3998951206.py:1)
24/03/01 12:54:24 INFO DAGScheduler: Parents of final stage: List()
24/03/01 12:54:24 INFO DAGScheduler: Missing parents: List()
24/03/01 12:54:24 INFO DAGScheduler: Submitting ShuffleMapStage 4 (MapPartitionsRDD[10] at collect at /tmp/ipykernel_56411/3998951206.py:1), which has no missing parents
24/03/01 12:54:24 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 36.3 KiB, free 413.9 MiB)
24/03/01 12:54:24 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 15.5 KiB, free 413.9 MiB)
24

Average Values::
avg(duration): 236.8841377784466
avg(tempo): 125.78977758234197
avg(energy): 0.0
avg(danceability): 0.0
avg(loudness): -10.435855814655746


# STOP SPARK

In [73]:
# release the cores
spark_context.stop()

24/03/08 14:36:36 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/03/08 14:36:36 INFO SparkUI: Stopped Spark web UI at http://master-node:4040
24/03/08 14:36:36 INFO YarnClientSchedulerBackend: Interrupting monitor thread
24/03/08 14:36:36 INFO YarnClientSchedulerBackend: Shutting down all executors
24/03/08 14:36:36 INFO YarnSchedulerBackend$YarnDriverEndpoint: Asking each executor to shut down
24/03/08 14:36:36 INFO YarnClientSchedulerBackend: YARN client scheduler backend Stopped
24/03/08 14:36:36 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/03/08 14:36:36 INFO MemoryStore: MemoryStore cleared
24/03/08 14:36:36 INFO BlockManager: BlockManager stopped
24/03/08 14:36:36 INFO BlockManagerMaster: BlockManagerMaster stopped
24/03/08 14:36:36 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/03/08 14:36:36 INFO SparkContext: Successfully stopped SparkContext
